<a href="https://colab.research.google.com/github/Lorddickenstein/FSLRwithNLP/blob/main/Application/CNN_Model_Final_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network with Own Datasets #1

This is training half of the datasets from the folder FSLR Application Dataset. This uses the datasets under the Training, Valid, and Test folders but only half the data from the datasets.

The datasets provided are:


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Activation, Dense, Flatten, MaxPool2D, Conv2D, Dropout, BatchNormalization
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2
import glob
import warnings
import random
import shutil
import itertools

warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [56]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

## Do you have GPU?
If using a GPU, run this to make the computer know that you have a gpu so tensorflow could identify it correctly and enable memory growth on the gpu.

Otherwise, leave this code.

In [ ]:
physical_devices = tf.config.expertimental.list_physical_devices('GPU')
print('Nump GPUs Available: ', len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Data Preparation
The images are found inside the OurDataset Folder under Raw_Dataset. Images will be read by the program and the output will be placed into the inside the FSLR_Application_Dataset.

In [30]:
# paths and directories
root = '/content/drive/MyDrive/Colab Notebooks/Datasets/Part 1'
dataset_root = '/content/drive/MyDrive/Colab Notebooks/Datasets/FSLR_Application_Dataset'

train_path = os.path.join(root, "Train")
valid_path = os.path.join(root, "Valid")
test_path = os.path.join(root, "Test")

In [20]:
# Organize data into train, valid, test dirs
dataset_categories = ['Dynamic Single', 'Dynamic Double',
                       'Static Single', 'Static Double',
                       'Letters', 'Numbers']

letters = ['A', 'B', 'C', 'D', 'E',
           'F', 'G', 'H', 'I', 'J',
           'K', 'L', 'M', 'N', 'O',
           'P', 'Q', 'R', 'S', 'T',
           'U', 'V', 'W', 'X', 'Y',
           'Z']

# numbers = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
numbers = []

static_single = ['Fine', 'Gabi', 'Good', 'Hapon', 'He-She',
                 'His-Her', 'I Love You', 'I-Me', 'Mine',
                 'Tanghali', 'Umaga', 'You', 'Your']

static_double = ['Congratulations', 'Great', 'Help', 'Meet', 'Table',
                 'Name', 'Occupation', 'Pray', 'Rest', 'Pen',
                 'Stand', 'Study', 'To', 'Chair']

dynamic_single = ['Invite', 'Late', 'Late-2',
                  'No', 'No-2', 'Our', 'Our-2', 'Sorry', 'That', 'Them',
                  'This', 'We', 'Welcome', 'Welcome-2', 'When', 'Who',
                  'Who-2', 'Why', 'Why-2', 'Yes', 'Yesterday']

dynamic_double = ['Ball', 'Banana', 'Banana-2', 'Bread', 'Break', 'Egg',
                  'Break-2', 'Bring', 'Bring-2', 'Buy', 'Buy-2', 'Bye', 'Coconut', 'Egg-2',
                  'Coffee', 'Come', 'Come-2', 'Cook', 'From', 'From-2', 'Get', 'Get-2', 'Egg-3',
                  'Go', 'Go-2', 'Happen', 'Happen-2', 'How', 'How-2', 'Introduce', 'Introduce-2',
                  'Let', 'Let-2', 'Live', 'Mango', 'Maybe', 'Nice', 'Now', 'Office', 'Office-2',
                  'School', 'Sit', 'Store', 'Strawberry', 'Thank You', 'Thank You-2', 'Today', 'Today-2',
                  'What', 'Where', 'Which', 'Work']

"""
  Removed:
    eroplano
    eroplano-2
    numbers[]
    year
    allow
    ago
    how much
    old
"""

dataset_classes = numbers + letters + static_single + static_double + dynamic_single + dynamic_double

In [ ]:
# Run this to take only half of the dataset classes
dataset_classes = dataset_classes[:63]
print(dataset_classes)

In [ ]:
print(len(letters))
print(len(numbers))
print(len(static_single))
print(len(static_double))
print(len(dynamic_single))
print(len(dynamic_double))
print(len(dataset_classes))

## Functions
Contains all the functions that are needed for this program.

In [12]:
# Normalize images
def preprocess_func(src_img):
  norm = src_img.astype('float32')
  norm /= 255
  return norm

In [13]:
# Plot 10 sample images
def plotImages(images_arr):
  fig, axes = plt.subplots(1, 10, figsize=(20, 20))
  axes = axes.flatten()
  for img, ax in zip(images_arr, axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()

In [14]:
# Create the Sequential Model
def create_model():
  model = Sequential()

  # Layers
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(80, 80, 1), padding='same'))
  model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPool2D(pool_size=(2, 2), strides=2))
  model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPool2D(pool_size=(2, 2), strides=2))
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.20))
  model.add(Dense(total_classes, activation='softmax'))

  return model

In [15]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Populate **Train**, **Valid**, and **Test** Folders
Populate the **Train, Valid, and Test** folder with directories of classes. Uses ***random sampling*** to randomize selection of images and uses ***glob*** to select file paths matching specific pattern in their names. ***shutil*** module allows the transfer of these files to the *Train, Valid, and Test*

In [ ]:
folders = ['Train', 'Valid', 'Test']
for folder in folders:
  folder_path = os.path.join(root, folder)
  dataset_path = os.path.join(dataset_root, folder)
  for sign in dataset_classes:
    path_class_dest = os.path.join(folder_path, sign)
    print(path_class_dest)

    if os.path.isdir(path_class_dest) is False:
      os.makedirs(path_class_dest)

    path_class = os.path.join(dataset_path, sign)
    for item in os.listdir(path_class):
      shutil.copy(os.path.join(path_class, item), path_class_dest)
      print(folder, '\t', sign,'\t', item)


In [ ]:
# Verify number of images in all folders
for sign in dataset_classes:
  train_path_experiment = os.path.join(train_path, sign)
  valid_path_experiment = os.path.join(valid_path, sign)
  test_path_experiment = os.path.join(test_path, sign)
  print(sign, "\t\t\t\t", len(os.listdir(train_path_experiment)), len(os.listdir(valid_path_experiment)), len(os.listdir(test_path_experiment)))

# Preprocess Image
Transform images from the dataset into a format that the model expect. Applies data augmentation to increase the number of datasets used for training.

In [44]:
image_size = (80, 80)

In [45]:
# Augments dataset 10x
train_batches = ImageDataGenerator(preprocessing_function=preprocess_func, horizontal_flip=True, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, fill_mode='nearest') \
    .flow_from_directory(directory=train_path, target_size=image_size, classes=dataset_classes, batch_size=5, color_mode='grayscale')
valid_batches = ImageDataGenerator(preprocessing_function=preprocess_func, horizontal_flip=True, width_shift_range=0.15, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, fill_mode='nearest') \
    .flow_from_directory(directory=valid_path, target_size=image_size, classes=dataset_classes, batch_size=5, color_mode='grayscale')
test_batches = ImageDataGenerator(preprocessing_function=preprocess_func, horizontal_flip=True, width_shift_range=0.15, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, fill_mode='nearest') \
    .flow_from_directory(directory=test_path, target_size=image_size, classes=dataset_classes, batch_size=5, color_mode='grayscale')

Found 9909 images belonging to 63 classes.
Found 3100 images belonging to 63 classes.
Found 1240 images belonging to 63 classes.


In [ ]:
assert valid_batches.n == 6950
assert test_batches.n == 2780

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
plotImages(imgs)

# Save checkpoints during training
##Employing the following:

1. Checkpoints

2. CSV Logger

In [46]:
from keras.callbacks import ModelCheckpoint, CSVLogger

checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/Datasets/Part 1/weights_improvements-epoch:{epoch:02d}-val_accuracy:{val_accuracy:.2f}.hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = ModelCheckpoint(checkpoint_path,
                              verbose=1,
                              monitor='val_accuracy',
                              mode='max',
                              save_best_only=True,
                              save_weights_only=True)

log_folder = '/content/drive/MyDrive/Colab Notebooks/Datasets/Part 1'
log_path = os.path.join(log_folder, 'FSLR_logs.csv')
log_csv = CSVLogger(log_path, separator=',', append=False)

callback_list = [cp_callback, log_csv]

# Building the model

Model #1 to create a new model from scratch.

Model #2 to resume the training for when the training is disrupted, stopped or first set of epoch finished. Training can continue in another time.

In [48]:
# Count the total classes that the model must know
total_classes = len(os.listdir(train_path))
print(total_classes)
assert len(dataset_classes) == total_classes

63


In [49]:
batch_size = 200
epochs = 10

## Model #1 New Model
Choose only one model.

In [50]:
# Create Model function
model = create_model()

In [51]:
# Summary of layers
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 80, 80, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 80, 80, 128)       73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 80, 80, 128)       147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 40, 40, 256)       295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 40, 40, 256)       590080    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 256)       0

In [52]:
# Compile the layers into one model and create a connection
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
# Train the model with the new callback
history = model.fit(x=train_batches,
                    validation_data=valid_batches,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=callback_list)

Epoch 1/10
1982/1982 [==============================] - 5333s 3s/step - loss: 4.1250 - accuracy: 0.0152 - val_loss: 4.1326 - val_accuracy: 0.0161

Epoch 00001: val_accuracy improved from -inf to 0.01613, saving model to /content/drive/MyDrive/Colab Notebooks/Datasets/Part 1/weights_improvements-epoch:01-val_accuracy:0.02.hdf5
Epoch 2/10
 207/1982 [==>...........................] - ETA: 1:15:00 - loss: 4.1256 - accuracy: 0.0213

## Model #2 Resume Training
Choose only one model

In [ ]:
# Model configuration
new_model_name = '' # must be in this format: 'FSLR_CNN_Model(02-epochs)-accuracy:0.00-val_accuracy:0.00.h5'
new_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/CNN Model'
new_model_path = os.path.join(new_path, new_model_name)

In [ ]:
# Load Model
new_model = load_model(new_model_path)

In [ ]:
# Test model before resuming training
print(new_model.evaluate(test_batches, verbose=0))

In [ ]:
# Create the connection and train the model
new_model.fit(x=train_batches,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=valid_batches,
          callbacks=callback_list)

# Evaluate the model with test_sets
print(new_model.evaluate(test_batches))

In [ ]:
new_model.save(new_model_path)

# Save the model

In [ ]:
# Model configurations
model_name = 'FSLR_CNN_Model(00-epochs)-accuracy:0.00-val_accuracy:0.00.h5'
path = '/content/drive/MyDrive/Colab Notebooks/Datasets/CNN Model'
model_path = os.path.join(path, model_name)

In [ ]:
# save the model
model.save(model_path)

# Confusion Matrix
Plot the confusion matrix.

In [ ]:
# Create a prediction
predictions = model.predict(x=test_batches, verbose=0)

In [ ]:
# Setup the confusion matrix
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
# Check class indices
test_batches.class_indices

In [ ]:
# Plot the confusion matrix
cm_plot_labels = total_classes
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [55]:
import os
from os import listdir
from PIL import Image

categ = ['Train', 'Valid', 'Test']
dataset = '/content/drive/MyDrive/Colab Notebooks/Datasets/FSLR_Application_Dataset'

for cat in categ:
  img_path = os.path.join(dataset, cat)
  for foldername in listdir(img_path):
    sign_path = os.path.join(img_path, foldername)
    print(sign_path)
    for sign in listdir(sign_path):
      if sign.endswith('.jpg'):
        try:
          img = Image.open(os.path.join(sign_path, sign)) # open the image file
          img.verify() # verify that it is, in fact an image
        except (IOError, SyntaxError, OSError) as e:
          print('Bad file:', sign) # print out the names of corrupt files

/content/drive/MyDrive/Colab Notebooks/Datasets/FSLR_Application_Dataset/Train/A
/content/drive/MyDrive/Colab Notebooks/Datasets/FSLR_Application_Dataset/Train/B
/content/drive/MyDrive/Colab Notebooks/Datasets/FSLR_Application_Dataset/Train/C
/content/drive/MyDrive/Colab Notebooks/Datasets/FSLR_Application_Dataset/Train/D
/content/drive/MyDrive/Colab Notebooks/Datasets/FSLR_Application_Dataset/Train/E
/content/drive/MyDrive/Colab Notebooks/Datasets/FSLR_Application_Dataset/Train/F
/content/drive/MyDrive/Colab Notebooks/Datasets/FSLR_Application_Dataset/Train/G
/content/drive/MyDrive/Colab Notebooks/Datasets/FSLR_Application_Dataset/Train/H
/content/drive/MyDrive/Colab Notebooks/Datasets/FSLR_Application_Dataset/Train/I
/content/drive/MyDrive/Colab Notebooks/Datasets/FSLR_Application_Dataset/Train/J
/content/drive/MyDrive/Colab Notebooks/Datasets/FSLR_Application_Dataset/Train/K
/content/drive/MyDrive/Colab Notebooks/Datasets/FSLR_Application_Dataset/Train/L
/content/drive/MyDrive/Colab